# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('log_reg').getOrCreate()

In [4]:
df = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [7]:
# account manager and churn are binary columns in the dataset
df[['Churn']].distinct().show()
df[['Account_Manager']].distinct().show()

+-----+
|Churn|
+-----+
|    1|
|    0|
+-----+

+---------------+
|Account_Manager|
+---------------+
|              1|
|              0|
+---------------+



In [8]:
column_names = df.columns

In [9]:
# checking for nulls

from pyspark.sql.functions import col, filter

for column in column_names:
    print(column)
    df_nulls = df.filter(col(column).isNull())
    df_nulls.show()

Names
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+

Age
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+

Total_Purchase
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+------------

In [10]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, VectorIndexer

In [13]:
# Checking distinct categories
df[['Location']].distinct().show()
df[['Company']].distinct().show()

+--------------------+
|            Location|
+--------------------+
|062 Trevor Falls ...|
|066 Jenkins Walks...|
|45946 Day Springs...|
|143 Andrea Flat L...|
|Unit 2093 Box 153...|
|399 Herbert Key P...|
|104 Ruben Rapid A...|
|930 Carrie Harbor...|
|8202 Jade Unions ...|
|USCGC Bailey FPO ...|
|893 Carla Trace S...|
|446 Rodney Ridge ...|
|30668 Isabella Fr...|
|911 Kent Point An...|
|078 Nunez Haven S...|
|PSC 5667, Box 831...|
|4972 Michael Vill...|
|567 Ian Loop Lamb...|
|482 Wells Mountai...|
|7259 Brown Street...|
+--------------------+
only showing top 20 rows

+--------------------+
|             Company|
+--------------------+
|Miller, Johnson a...|
|Hunter, Reyes and...|
|          Obrien PLC|
|            Soto PLC|
|            Todd LLC|
|Smith, Marshall a...|
|           Smith PLC|
|          Hall Group|
|Freeman, Lam and ...|
|       Smith-Carroll|
|Hall, Hernandez a...|
|          Cannon Inc|
|        White-Dennis|
|Wilson, Collins a...|
|Jennings, Gates a...|
|     Ca

In [19]:
# Checking distinct categories
df[['Location']].distinct().count()

900

In [20]:
# Checking distinct categories
df[['Company']].distinct().count()

873

In [24]:
# Checking distinct categories
df[['Names']].distinct().count()

899

In [22]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [38]:
from pyspark.ml.classification import LogisticRegression

In [47]:
assembler = VectorAssembler(inputCols=[
    'Age',
    'Total_Purchase',
    'Account_Manager',
    'Years',
    'Num_Sites'
], outputCol='features')

In [48]:
assemble_df = assembler.transform(df)

In [49]:
assemble_df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

In [50]:
final_df = assemble_df.select("features", "Churn")

In [51]:
train_data, test_data = final_df.randomSplit([0.7, 0.3])

In [52]:
log_reg_churn = LogisticRegression(featuresCol='features', labelCol='Churn')

In [53]:
fit_model=log_reg_churn.fit(train_data)

In [54]:
results = fit_model.transform(test_data)

In [55]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator 

In [56]:
evaluation = BinaryClassificationEvaluator(rawPredictionCol= 'prediction',
                                           labelCol = "Churn")

In [57]:
results.select('Churn', "prediction").show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [59]:
AUC = evaluation.evaluate(results)

In [60]:
AUC

0.760981308411215